In [1]:
import pandas as pd
import os

In [ ]:
# 23년도 데이터로 매핑할 dictionary 만들기
# 행정동 있는 데이터 경로 아무거나
df_mapping = pd.read_csv(r"C:\Users\eunyo\Desktop\FOM\environment contest\기온_습도_미세먼지\S-DoT_NATURE_2023년\S-DoT_NATURE_2023.01.02-01.08.csv", encoding="cp949", index_col=False)

df_mapping = df_mapping.drop_duplicates(subset="시리얼", keep="first") # 시리얼 unique인 값만

mapping = dict(zip(df_mapping["시리얼"], df_mapping["행정동"])) # 매핑할 dict 생성

# 행정동 이름이 정수형으로 되어있는것 제거
mapping = {s:int(n) if n.lstrip("-").isdigit() else n for s,n in mapping.items()}
mapping = {s:n for s,n in mapping.items() if not isinstance(n,int)}

In [ ]:
# 간단한 전처리 함수
def preprocessing_2022(df):
    # df에 매핑
    df["행정동"] = df["시리얼"].replace(mapping)

    #필요한 열들로 df 재구성
    keep = ["행정동", "기온(℃)", "상대습도( %)", "초미세먼지 보정(㎍/㎥)", "미세먼지 보정(㎍/㎥)", "등록일자"]
    df = df[keep]

    # 열들 이름, 위치 변경
    df = df.rename(columns={"기온(℃)":"기온", "상대습도( %)":"상대습도", "초미세먼지 보정(㎍/㎥)":"초미세먼지", "미세먼지 보정(㎍/㎥)":"미세먼지"})
    df = df[["등록일자", "행정동", "기온", "상대습도", "초미세먼지", "미세먼지"]]

    # df["등록일자"] = pd.to_datetime(df["등록일자"]).dt.strftime("%Y-%m-%d") # 날짜 형식 변경
    def fun(data):
        data = str(data).split()[0]
        return data

    df["등록일자"] = df['등록일자'].apply(fun)

    return df

def preprocessing_2023(df):
    keep = ["행정동", "온도 평균(℃)", "습도 평균(%)", "초미세먼지 평균(㎍/㎥)", "미세먼지 평균(㎍/㎥)", "등록일시"]
    df = df[keep]

    df = df.rename(columns={"온도 평균(℃)":"기온", "습도 평균(%)":"상대습도", "초미세먼지 평균(㎍/㎥)":"초미세먼지", "미세먼지 평균(㎍/㎥)":"미세먼지", "등록일시":"등록일자"})
    df = df[["등록일자", "행정동", "기온", "상대습도", "초미세먼지", "미세먼지"]]

    def fun(data):
        data = str(data).split()[0]
        return data

    df["등록일자"] = df['등록일자'].apply(fun)

In [ ]:
def preprocessing(data, file_name):
    global use
    try:
        if use:
            data = preprocessing_2022(data)
        else:
            data = preprocessing_2023(data)

    except KeyError: # ParserError 발생하면 해당 행 무시
        if use:
            use = False
            print(f"\"{file_name}\" 에서 KeyError 발생")
            print("전처리 방식 변경(2022 -> 2023)")
            data = preprocessing_2023(data)
        else:
            use = True
            print(f"\"{file_name}\" 에서 KeyError 발생")
            print("전처리 방식 변경(2023 -> 2022)")
            data = preprocessing_2022(data)

    return data

In [ ]:
def safe_read(file_path, file_name):
    try:
        df = pd.read_csv(file_path, encoding="cp949", index_col=False)

    except pd.errors.ParserError: # ParserError 발생하면 해당 행 무시
        print(f"\"{file_name}\" 에서 ParserError 발생")
        df = pd.read_csv(file_path, encoding="cp949", index_col=False, on_bad_lines="skip") 

    except UnicodeDecodeError: # UnicodeDecodeError 발생하면 인코딩 방식 변경
        print(f"\"{file_name}\" 에서 UnicodeDecodeError 발생")
        df = pd.read_csv(file_path, encoding="utf8", index_col=False)
        
    return df

In [ ]:
input = r"C:\Users\eunyo\Desktop\FOM\environment contest\기온_습도_미세먼지\S-DoT_NATURE_2022년" # 입력 파일들 폴더 경로
output = r"C:\Users\eunyo\Desktop\FOM\environment contest\기온_습도_미세먼지\전처리\2022" #출력 파일 저장할 폴더 경로

files = os.listdir(input)

dfs = []
use = True

for file in files:
    input_path = os.path.join(input, file) # 입력 파일 경로 설정
    
    # 파일 가져오기 # 열이 갑자기 많아져서 에러 발생 -> 그 행 무시
    df = safe_read(input_path, file)

    df = preprocessing(df, file) # 전처리

    dfs.append(df)
    
    # # 개별로 저장
    # ouput_path = os.path.join(output, file) # 출력 파일 경로 설정
    # df.to_csv(ouput_path, encoding="utf8", index=False) # 파일 저장

result_df = pd.concat(dfs, ignore_index=True) # 파일 하나로 합치기

ouput_path = os.path.join(output, "result_2022.csv") # 출력 파일 경로 설정
result_df.to_csv(ouput_path, encoding="utf8", index=False) # 파일 저장

In [ ]:
# 용량이 너무 커서 4등분 함
df = pd.read_csv(r"C:\Users\eunyo\Desktop\FOM\environment contest\기온_습도_미세먼지\전처리\2022\result_2022.csv")

output = r"C:\Users\eunyo\Desktop\FOM\environment contest\기온_습도_미세먼지\전처리\2022"

length = len(df)
quarter = length // 4

for i in range(4):
    res = df.iloc[quarter*i:quarter*(i+1)]

    ouput_path = os.path.join(output, f"result_2022_{i+1}.csv") # 출력 파일 경로 설정
    res.to_csv(ouput_path, encoding="utf8", index=False) # 파일 저장